# 9 Synthetic Control - CPS Data
Julian Hsu
4apr2022

Create state-month level CPS data.

In [14]:
import pandas as pd
import numpy as np
import os as os 

from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib inline  

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.conditional_models import ConditionalLogit

from IPython.display import display    


import scipy.stats 

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error


In [15]:
os.getcwd()

'/Users/hsujulia/Downloads/CPSData'

CPS data are downloaded on local drive. You can find the CPS files from here: https://data.nber.org/cps-basic2/docs/

Data codebook is here: https://data.nber.org/cps-basic2/docs/cpsb202001.ddf

Need to know:

| variable name | description |
| :--- | :--- |
| GESTFIPS | state |
| PWSSWGT  | final weight |
| HRMONTH  | Month of data|
| HRYEAR4  | Year of data |
| HEFAMINC | Family Income bracket |
| PRERNWA  | Weekly earnings| 
| PRTAGE   | Age| 
| PEMLR    | MONTHLY LABOR FORCE | 
| PEERNHRY | Whether earnings are hourly | 
| PUERNH1C | hourly wage |
| PRWKSTAT | Part-time full time status| 
| PERET1   | Eligibiligy (whether want a part-time job)| 
| HRHTYPE  | Married| 
| HETENURE | Living quarters are rent| 
| PEEDUCA  | Highest level of school completed| 
| PTDTRACE | Race| 
| PENATVTY | Country of birth| 
| PRAGNA   | Worked in agriculture| 
 


In [16]:
os.chdir('/Users/hsujulia/Downloads/CPSData')

from os import listdir
from os.path import isfile, join
cps_files = [f for f in listdir(os.getcwd()) if isfile(join(os.getcwd(), f)) and 'cps' in f]

column_list = [
'GESTFIPS',
'PWSSWGT',
'HRMONTH',
'HRYEAR4',
'HEFAMINC',
'PRERNWA',
'PRTAGE',
'PEMLR',
'PEERNHRY',
'PUERNH1C', 
'PRWKSTAT',
'PERET1',
'HRHTYPE',
'HETENURE',
'PEEDUCA',
'PTDTRACE',
'PENATVTY',
'PRAGNA']
column_list = [x.lower() for x in column_list]


In [17]:
df_final_output = pd.DataFrame()

In [18]:
for c in cps_files:
    df = pd.read_csv(c)[column_list]
    ## Replace negative numbers with missing
    df.replace(to_replace=[-2,-1], value=np.NaN, inplace=True)

    ## State    
    df['state'] = df['gestfips'].replace( {1:'AL', 30:'MT',
     2:'AK', 31:'NE',
     4:'AZ', 32:'NV',
     5:'AR', 33:'NH',
     6:'CA', 34:'NJ',
     8:'CO', 35:'NM',
     9:'CT', 36:'NY',
    10:'DE', 37:'NC',
    11:'DC', 38:'ND',
    12:'FL', 39:'OH',
    13:'GA', 40:'OK',
    15:'HI', 41:'OR',
    16:'ID', 42:'PA',
    17:'IL', 44:'RI',
    18:'IN', 45:'SC',
    19:'IA', 46:'SD',
    20:'KS', 47:'TN',
    21:'KY', 48:'TX',
    22:'LA', 49:'UT',
    23:'ME', 50:'VT',
    24:'MD', 51:'VA',
    25:'MA', 53:'WA',
    26:'MI', 54:'WV',
    27:'MN', 55:'WI',
    28:'MS', 56:'WY',
    29:'MO' } )   
    
    
    ## Labor Force Recode
    x = df['pemlr'].value_counts().sort_index()
    ###print(x)

    df['laborforce_employed'] = df['pemlr'].isin([1,2])
    df['laborforce_unemployed'] = df['pemlr'].isin([3,4])
    df['laborforce_nopart'] = df['pemlr'].isin([5,6,7])

    ## Family Income Brackets
    x = df['hefaminc'].value_counts().sort_index()
    ###print(x.cumsum() / x.sum() )

    df['hefaminc_0k_15k'] = df['hefaminc'].between(1,5)
    df['hefaminc_15k_50k'] = df['hefaminc'].between(6,11)
    df['hefaminc_50k_100k'] = df['hefaminc'].between(12,14)
    df['hefaminc_100kplus'] = df['hefaminc'].between(15,99)

    ## Hourly Status
    df['peernhry'] = df['peernhry'].replace({1:1, 2:0},inplace=False)
    ###print(df['peernhry'].value_counts().sort_index())

    ## Parttime status
    df['laborforce_no'] = (df['prwkstat']==1)
    df['laborforce_unemp'] = df['prwkstat'].isin([11,12])
    df['laborforce_ft'] = df['prwkstat'].isin([2,3,4,5])
    df['laborforce_pt'] = df['prwkstat'].isin([6,7,8,9,10])

    ## Marriage status
    df['marriage'] = df['hrhtype'].isin([1,2])

    ## Rent
    df['rent'] = df['hetenure'].isin([2,3])

    ## Agricultural Workers
    df['agri'] = df['pragna'].replace({1:1, 2:0},inplace=False)


    ## Schooling
    x = df['peeduca'].value_counts().sort_index()
    ###print(x.cumsum() / x.sum() )
    df['educ_lesshs'] = df['peeduca'].isin([31,32,33,34,35,36,37,38,39])
    df['educ_someAA'] = df['peeduca'].isin([40,41,42])
    df['educ_BA'] = df['peeduca'].isin([43])
    df['educ_grad'] = df['peeduca'].isin([44,45,46])

    # df[['prernwa', 'puernh1c']].describe()
    stats_list = [       'laborforce_employed', 'laborforce_unemployed', 'laborforce_nopart',
           'hefaminc_0k_15k', 'hefaminc_15k_50k', 'hefaminc_50k_100k',
           'hefaminc_100kplus', 'laborforce_no', 'laborforce_unemp',
           'laborforce_ft', 'laborforce_pt', 'marriage', 'rent', 'agri',
           'educ_lesshs', 'educ_someAA', 'educ_BA', 'educ_grad', 'prernwa']


    ## weight denominator
    weight_total = df.groupby(['state','hrmonth','hryear4'])['pwsswgt'].sum()
    for s in stats_list:
        df[s] = df[s]*df['pwsswgt']

    df_final = df.groupby(['state','hrmonth','hryear4'])[stats_list].sum()


    df_final = df_final.join(weight_total)
    for s in stats_list:
        df_final[s] = df_final[s].divide(df_final['pwsswgt'])

    df_final_output = df_final_output.append(df_final)

In [19]:
df_final_output.to_pickle('/Users/hsujulia/Documents/GitHub/causalinference_crashcourse/Notebooks/9 Synthetic Control - CPS Data')